In [75]:
import collections
from collections import Iterable, ByteString
from contextlib import ExitStack
from enum import Enum
import glob
from io import BytesIO
from itertools import groupby, islice
import os

import py7zlib

def namedtuple_with_defaults(typename, field_names, defaults=()):
    T = collections.namedtuple(typename, field_names)
    T.__new__.__defaults__ = (None,) * len(T._fields)
    if isinstance(defaults, collections.Mapping):
        prototype = T(**defaults)
    else:
        prototype = T(*defaults)
    T.__new__.__defaults__ = tuple(prototype)
    return T

namedtuple = namedtuple_with_defaults

class ABCModality(Enum):
    META = 'meta'
    PARA = 'para'
    STEP = 'step'
    STL2 = 'stl2'
    OBJ = 'obj'
    FEAT = 'feat'
    STAT = 'stat'


ALL_ABC_MODALITIES = [mod for mod in ABCModality]
ABC_7Z_FILEMASK = 'abc_{chunk}_{modality}_v{version}.7z'

# def _compose_filemask(chunks, modalities, version):

#     chunk_mask = '*'
#     if None is not chunks:
#         assert isinstance(chunks, Iterable)
#         assert all(isinstance(s, ByteString) for s in chunks)

#         chunk_mask = '[' + '|'.join(chunks) + ']'

#     modalities

#     version

#     return ABC_7Z_FILEMASK.format(
#         chunk=chunk_mask, modality='x', version=0,
#     )


def _extract_modality(filename):
    name = os.path.basename(filename)
    name, ext = os.path.splitext(name)
    abc, chunk, modality, version = name.split('_')
    return modality


# ABCItem is the primary data instance in ABC, containing all the modalities
ABCItem = namedtuple(
    'ABCItem',
    'pathname archive_pathname ' + ' '.join(modality.value for modality in ALL_ABC_MODALITIES))


class ABC7ZFile(object):
    """A helper class for reading 7z files."""

    def __init__(self, filename):
        self.filename = filename
        self.modality = _extract_modality(filename)
        self._reset_handles()
    
    def _reset_handles(self):
        self.file_handle = None
        self.archive_handle = None
        self._names_list = None        

    def _open(self):
        self.file_handle = open(self.filename, 'rb')
        self.archive_handle = py7zlib.Archive7z(self.file_handle)
        self._names_list = self.archive_handle.getnames()

    def _close(self):
        self.file_handle.close()
        self._reset_handles()

    def __enter__(self):
        self._open()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self._isopen():
            self._close()
            
    def _get_item_by_name(self, name):
        bytes_io = BytesIO(self.archive_handle.getmember(name).read())
        return ABCItem(self.filename, name, **{self.modality: bytes_io})

    def __iter__(self):
        for name in self.archive_handle.getnames():
            bytes_io = BytesIO(self.archive_handle.getmember(name).read())
            yield ABCItem(self.filename, name, **{self.modality: bytes_io})

    def _isopen(self):
        return (self.file_handle is not None and
                self.archive_handle is not None and
                self._names_list is not None)
    
    def __getitem__(self, key):
        assert self._isopen()
        print(key)
        
        if isinstance(key, int):
            name = self._names_list[key]
            return self._get_item_by_name(name)
        elif isinstance(key, slice):
            names = list(islice(self._names_list, key.start, key.stop, key.step))
            return (self._get_item_by_name(name) for name in names)



# class ABCChunk(Iterable):
#     """A chunk is a collection of files (with different modalities),
#     that iterates over all files simultaneously."""

#     def __init__(self, filenames):
#         self.filenames = filenames
#         self.file_handles = []

#     def __iter__(self):
#         with ExitStack() as stack:
#             self.file_handles = [stack.enter_context(ABC7ZFile(filename))
#                                  for filename in self.filenames]


#         for filename, handle in zip(self.filenames, self.file_handles):


#         item = ABCItem()

#         yield item



# class ABCData(Iterable):
#     def __init__(self, data_dir, modalities=ALL_ABC_MODALITIES, chunks=None,
#                  shape_representation='trimesh', version='00'):
#         self.data_dir = data_dir
#         self.modalities = modalities
#         self.chunks = chunks
#         self.version = version
#         self.shape_representation = shape_representation

#         filemask = _compose_filemask(self.chunks, self.modalities, self.version)

#         self.data_files = glob.glob(os.path.join(self.data_dir, filemask))

#         chunk_getter = lambda s: ABC_7Z_FILEMASK.

#         self.data_files = {chunk: chunk_files
#                            for chunk, chunk_files in groupby(self.data_files, key=chunk_getter)}

#     def __iter__(self):
#         for chunk in self.chunks:
#             filenames_by_chunk = self.data_files[chunk]
#             chunk = ABCChunk(**filenames_by_chunk)
#             for item in chunk:
#                 yield item


In [43]:
with ABC7ZFile('/home/artonson/tmp/abc/abc_0000_obj_v00.7z') as abc_7z_file:
#     x = abc_7z_file[0:3]
    
    print(len(abc_7z_file.archive_handle.getnames()))

7168


In [42]:
x

[ABCItem(pathname='/home/artonson/tmp/abc/abc_0000_obj_v00.7z', archive_pathname='00000002/00000002_1ffb81a71e5b402e966b9341_trimesh_001.obj', meta=None, para=None, step=None, stl2=None, obj=<_io.BytesIO object at 0x7f7c1254e468>, feat=None, stat=None),
 ABCItem(pathname='/home/artonson/tmp/abc/abc_0000_obj_v00.7z', archive_pathname='00000003/00000003_1ffb81a71e5b402e966b9341_trimesh_002.obj', meta=None, para=None, step=None, stl2=None, obj=<_io.BytesIO object at 0x7f7c13fe1ca8>, feat=None, stat=None),
 ABCItem(pathname='/home/artonson/tmp/abc/abc_0000_obj_v00.7z', archive_pathname='00000004/00000004_1ffb81a71e5b402e966b9341_trimesh_003.obj', meta=None, para=None, step=None, stl2=None, obj=<_io.BytesIO object at 0x7f7c13fe1bf8>, feat=None, stat=None)]

In [86]:
from joblib import Parallel, delayed

In [87]:
def read_slice(filename, slice_params):
    slice_start, slice_end = slice_params
    with ABC7ZFile(filename) as abc_7z_file:
        x = abc_7z_file[slice_start:slice_end]
        for i, item in enumerate(x):
            y = item.obj.getvalue()
            print(filename, slice_start + i, len(y))

In [88]:
read_slice(filename, (0, 2))

slice(0, 2, None)
/home/artonson/tmp/abc/abc_0000_obj_v00.7z 0 4523731
/home/artonson/tmp/abc/abc_0000_obj_v00.7z 1 4678073


In [89]:
slice_start = list(range(0, 7000, 100))
slice_end = list(range(99, 7099, 100))
slice_params = zip(slice_start, slice_end)

filename = '/home/artonson/tmp/abc/abc_0000_obj_v00.7z'

In [90]:
parallel = Parallel(n_jobs=40, verbose=10)
delayed_read_slice = delayed(read_slice)

parallel(
    delayed_read_slice(filename, sp) for sp in slice_params
)

[Parallel(n_jobs=40)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done   7 out of  70 | elapsed:  7.1min remaining: 64.0min
[Parallel(n_jobs=40)]: Done  15 out of  70 | elapsed: 10.5min remaining: 38.6min
[Parallel(n_jobs=40)]: Done  23 out of  70 | elapsed: 12.8min remaining: 26.2min
[Parallel(n_jobs=40)]: Done  31 out of  70 | elapsed: 13.5min remaining: 16.9min
[Parallel(n_jobs=40)]: Done  39 out of  70 | elapsed: 15.8min remaining: 12.6min
[Parallel(n_jobs=40)]: Done  47 out of  70 | elapsed: 16.8min remaining:  8.2min
[Parallel(n_jobs=40)]: Done  55 out of  70 | elapsed: 18.5min remaining:  5.0min
[Parallel(n_jobs=40)]: Done  63 out of  70 | elapsed: 19.7min remaining:  2.2min
[Parallel(n_jobs=40)]: Done  70 out of  70 | elapsed: 21.3min finished


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]